# Running CEDS Scenarios

In this notebook we document how to process and run data from the CEDS database.

In [1]:
%matplotlib inline

from os import listdir
from os.path import join, dirname
from pprint import pprint

import pandas as pd
import pyam
import pymagicc
from pymagicc.io import MAGICCData
import matplotlib.pyplot as plt
plt.style.use('bmh') 

import expectexception

<IPython.core.display.Javascript object>

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
TEST_DATA_PATH = join("..", "tests", "test_data")

## Reading in a CEDS csv

To read in CEDS csv's, we make use of the `pyam` library which is specifically designed for this purpose.

In [4]:
def read_ceds_csv(file_to_read):
    return pyam.IamDataFrame(
        data=file_to_read,
        encoding="utf-8"
    )

ceds_pyam_df = read_ceds_csv(join(TEST_DATA_PATH, "ceds-format-example.csv"))
ceds_pyam_df  # this just shows the type of ceds_pyam_df
ceds_pyam_df.data  # this returns the underlying DataFrame

INFO:root:Reading `../tests/test_data/ceds-format-example.csv`


,model,scenario,region,variable,unit,year,value
1138,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5ASIA,Emissions|BC,Mt BC/yr,2015,2
1229,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5LAM,Emissions|BC,Mt BC/yr,2015,5
1312,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5MAF,Emissions|BC,Mt BC/yr,2015,9
1395,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5OECD,Emissions|BC,Mt BC/yr,2015,2
1486,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5REF,Emissions|BC,Mt BC/yr,2015,10
1577,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,World,Emissions|BC,Mt BC/yr,2015,10
3414,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5ASIA,Emissions|BC,Mt BC/yr,2020,7
3505,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5LAM,Emissions|BC,Mt BC/yr,2020,9
3588,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5MAF,Emissions|BC,Mt BC/yr,2020,9
3671,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5OECD,Emissions|BC,Mt BC/yr,2020,8


## Checking an `IamDataFrame`

It is very easy to check that the sum of a given variable's sub-categories is equal to its declared total.

We show how in the next cell.

In [5]:
# show check_internal_consistency method here

## Converting IAM data to MAGICC data

The `pyam` library provides some very natural ways of filtering their DataFrames. These are detailed in their tutorial (TODO: add link). Here we use them to help convert IAM data into the emissions variables, regions and units used by MAGICC.

In [6]:
tdf = ceds_pyam_df.filter(
    level=1,
    model="MODEL-NAME-HYPHENS",
    scenario="SCENARIO-A-B-CDE-2",
    region="World",
)

In [7]:
def convert_ceds_to_magicc_variables(ceds_var):
    raw_ceds_var = ceds_var.replace("Emissions|", "")
    
    special_cases = {
        "Sulfur": "SOX",
    }
    
    if raw_ceds_var in special_cases:
        raw_var = special_cases[raw_ceds_var]
    else:
        raw_var = raw_ceds_var.replace("-", "").upper()
        
    return raw_var + "_EMIS"

def convert_ceds_to_openscm_world_table(pyam_df):
    # this function isn't useful for magicc as a magicc world table
    # needs CO2 to be split...
    # hence this is only useful for FAIR, HECTOR etc.
    magicc_df = pyam.IamDataFrame(data=pyam_df.data.copy())
    magicc_df = magicc_df.filter(
        level=1,
        region="World"
    )
    magicc_df.data.variable = magicc_df.data.variable.apply(convert_ceds_to_magicc_variables)
    magicc_df.data.region = magicc_df.data.region.str.upper()
    return magicc_df

In [8]:
convert_ceds_to_openscm_world_table(ceds_pyam_df).data

,model,scenario,region,variable,unit,year,value
1577,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,WORLD,BC_EMIS,Mt BC/yr,2015,10
3853,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,WORLD,BC_EMIS,Mt BC/yr,2020,4
6129,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,WORLD,BC_EMIS,Mt BC/yr,2030,4
8405,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,WORLD,BC_EMIS,Mt BC/yr,2040,4
10681,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,WORLD,BC_EMIS,Mt BC/yr,2050,6
12957,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,WORLD,BC_EMIS,Mt BC/yr,2060,1
15233,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,WORLD,BC_EMIS,Mt BC/yr,2070,4
17509,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,WORLD,BC_EMIS,Mt BC/yr,2080,9
19785,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,WORLD,BC_EMIS,Mt BC/yr,2090,1
22061,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,WORLD,BC_EMIS,Mt BC/yr,2100,2


In [9]:
# def convert_ceds_to_magicc_regional_table(pyam_df):
magicc_df = pyam.IamDataFrame(data=ceds_pyam_df.data.copy())
magicc_df = magicc_df.filter(
    region="World", 
    keep=False
).filter(
    level='1-', 
    keep=False
)
# magicc_df.data.variable = magicc_df.data.variable.apply(convert_ceds_to_magicc_variables)
magicc_df.data.region = magicc_df.data.region.str.upper()
odf = magicc_df.data[magicc_df.data.variable.isin([
    "Emissions|BC|Agricultural Waste Burning",
    "Emissions|BC|Grassland Burning",
])]
# magicc_df.variables()
odf.variable.unique()
odf
#     return magicc_df.data.copy()

array(['Emissions|BC|Agricultural Waste Burning',
       'Emissions|BC|Grassland Burning'], dtype=object)

,model,scenario,region,variable,unit,year,value
1139,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5ASIA,Emissions|BC|Agricultural Waste Burning,Mt BC/yr,2015,8
1230,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5LAM,Emissions|BC|Agricultural Waste Burning,Mt BC/yr,2015,7
1313,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5MAF,Emissions|BC|Agricultural Waste Burning,Mt BC/yr,2015,8
1396,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5OECD,Emissions|BC|Agricultural Waste Burning,Mt BC/yr,2015,8
1487,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5REF,Emissions|BC|Agricultural Waste Burning,Mt BC/yr,2015,2
3415,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5ASIA,Emissions|BC|Agricultural Waste Burning,Mt BC/yr,2020,1
3506,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5LAM,Emissions|BC|Agricultural Waste Burning,Mt BC/yr,2020,0
3589,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5MAF,Emissions|BC|Agricultural Waste Burning,Mt BC/yr,2020,5
3672,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5OECD,Emissions|BC|Agricultural Waste Burning,Mt BC/yr,2020,8
3763,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5REF,Emissions|BC|Agricultural Waste Burning,Mt BC/yr,2020,8


In [10]:
ceds_pyam_df.data

,model,scenario,region,variable,unit,year,value
1138,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5ASIA,Emissions|BC,Mt BC/yr,2015,2
1229,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5LAM,Emissions|BC,Mt BC/yr,2015,5
1312,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5MAF,Emissions|BC,Mt BC/yr,2015,9
1395,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5OECD,Emissions|BC,Mt BC/yr,2015,2
1486,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5REF,Emissions|BC,Mt BC/yr,2015,10
1577,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,World,Emissions|BC,Mt BC/yr,2015,10
3414,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5ASIA,Emissions|BC,Mt BC/yr,2020,7
3505,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5LAM,Emissions|BC,Mt BC/yr,2020,9
3588,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5MAF,Emissions|BC,Mt BC/yr,2020,9
3671,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5OECD,Emissions|BC,Mt BC/yr,2020,8


## Putting the checks together

We can now put all of this into a single class which will loop over all the models, scenarios, regions and variables and check if the sectoral and regional sum is equal to the declared totals. As our data is random, we expect that all of the sums will be incorrect except for the ones we just tried (which we adjusted to match by hand).

In [11]:
# class CedsDb(object):
#     def __init__(self, pyam_df):
#         self.pyam_df = pyam_df
#         self._model_scen_pyam_df = None
    
#     def check_sectoral_sums(self):
#         for (model, scenario), df in self.pyam_df.data.groupby(["model", "scenario"]):
#             self._check_sectoral_sums_model_scenario(model, scenario)
            
#     def _check_sectoral_sums_model_scenario(self, model, scenario):
#         self._model_scen_pyam_df = self.pyam_df.filter(
#             model=model,
#             scenario=scenario,
#         )
#         variables = self._model_scen_pyam_df.filter(
#             level=1
#         ).variables()
#         for variable in variables:
#             self._check_sectoral_sums_variable(variable)
            
#     def _check_sectoral_sums_variable(self, variable):
#         regions = self._model_scen_pyam_df.filter(
#             variable=variable,
#         ).regions()
#         for region in regions:
#             self._check_sectoral_sums_variable_region(variable, region)
    
#     def _check_sectoral_sums_variable_region(self, variable, region):
#         total_df = self._get_total_df_check_sectoral_sums_variable_region(variable, region)
        
#         model = total_df.columns.get_level_values("model")[0]
#         scenario = total_df.columns.get_level_values("scenario")[0]

#         sector_sum_df = self._get_sector_sum_df_check_sectoral_sums_variable_region(variable, region)
#         if sector_sum_df.empty:
#             print("No sectoral data for {}, {}, {}, {}".format(model, scenario, variable, region))
#         elif sector_sum_df.eq(total_df).all().all():
#             print("All good for {}, {}, {}, {}".format(model, scenario, variable, region))
#         else:
#             print("Sector sum does not match total for {}, {}, {}, {}".format(model, scenario, variable, region))
            
#     def _get_total_df_check_sectoral_sums_variable_region(self, variable, region):
#         return self._model_scen_pyam_df.filter(
#             variable=variable,
#             region=region,
#         ).pivot_table(
#             index=['year'], 
#             columns=['scenario', 'model', 'region', 'unit'], 
#             values='value',
#             aggfunc='sum', 
#         )
    
#     def _get_sector_sum_df_check_sectoral_sums_variable_region(self, variable, region):
#         return self._model_scen_pyam_df.filter(
#             variable='{}|*'.format(variable),
#             region=region,
#         ).pivot_table(
#             index=['year'], 
#             columns=['scenario', 'model', 'region', 'unit'], 
#             values='value',
#             aggfunc='sum', 
#         )
    
# ceds_db = CedsDb(ceds_pyam_df)

# ceds_db.check_sectoral_sums()

## Reshaping an `IamDataFrame`

Lastly we show how to reshape an `IamDataFrame` to get it into the format expected by `MAGICCData` so we can then write files with the data in it.

In [12]:
def reshape_pyam_df(pyam_df):
    raw_df = pyam_df.data
    
    reindexed_df = raw_df.set_index(
        ["model", "scenario", "region", "variable", "unit", "year"]
    ).unstack().T
    
    reindexed_df.index = reindexed_df.index.get_level_values("year")
    reindexed_df.index.name = "YEAR"
    
    models = reindexed_df.columns.get_level_values("model")
    scenarios = reindexed_df.columns.get_level_values("scenario")
    regions = reindexed_df.columns.get_level_values("region")
    variables = reindexed_df.columns.get_level_values("variable")
    units = reindexed_df.columns.get_level_values("unit")
    todos = ["SET"] * len(units)
    
    reindexed_df.columns = pd.MultiIndex.from_arrays(
        [models, scenarios, variables, todos, units, regions],
        names=("MODEL", "SCENARIO", "VARIABLE", "TODO", "UNITS", "REGION"),
    )
    
    return reindexed_df

In [13]:
reshape_pyam_df(ceds_pyam_df)

MODEL    MODEL-NAME-HYPHENS                                          \
SCENARIO SCENARIO-A-B-CDE-2                                           
VARIABLE       Emissions|BC Emissions|BC|Agricultural Waste Burning   
TODO                    SET                                     SET   
UNITS              Mt BC/yr                                Mt BC/yr   
REGION               R5ASIA                                  R5ASIA   
YEAR                                                                  
2015                      2                                       8   
2020                      7                                       1   
2030                      3                                       9   
2040                     10                                       8   
2050                      7                                       2   
2060                      0                                       9   
2070                      0                                       6   
2080                      2                                       1   
2090                     10                                       6   
2100                      8                                       8   

MODEL                                                            \
SCENARIO                                                          
VARIABLE Emissions|BC|Energy Sector Emissions|BC|Forest Burning   
TODO                            SET                         SET   
UNITS                      Mt BC/yr                    Mt BC/yr   
REGION                       R5ASIA                      R5ASIA   
YEAR                                                              
2015                              7                           5   
2020                              4                           2   
2030                              6                           1   
2040                              6                          10   
2050                              9                           8   
2060                              1                          10   
2070                              3                          10   
2080                              6                           0   
2090                              6                           4   
2100                              4                           4   

MODEL                                                                   \
SCENARIO                                                                 
VARIABLE Emissions|BC|Grassland Burning Emissions|BC|Industrial Sector   
TODO                                SET                            SET   
UNITS                          Mt BC/yr                       Mt BC/yr   
REGION                           R5ASIA                         R5ASIA   
YEAR                                                                     
2015                                  6                              1   
2020                                  7                              9   
2030                                  7                              4   
2040                                  4                              9   
2050                                  3                              0   
2060                                  2                              5   
2070                                  3                              0   
2080                                  4                              6   
2090                                  3                             10   
2100                                  1                              3   

MODEL                                                                         \
SCENARIO                                                                       
VARIABLE Emissions|BC|Peat Burning Emissions|BC|Residential Commercial Other   
TODO                           SET                                       SET   
UNITS                     Mt BC/yr                                  Mt BC

In [15]:
reshape_pyam_df(
    convert_ceds_to_openscm_world_table(ceds_pyam_df)
)

MODEL    MODEL-NAME-HYPHENS                                                \
SCENARIO SCENARIO-A-B-CDE-2                                                 
VARIABLE            BC_EMIS  C2F6_EMIS  CCL4_EMIS  CF4_EMIS   CFC113_EMIS   
TODO                    SET        SET        SET       SET           SET   
UNITS              Mt BC/yr kt C2F6/yr kt CCl4/yr kt CF4/yr kt CFC-113/yr   
REGION                WORLD      WORLD      WORLD     WORLD         WORLD   
YEAR                                                                        
2015                     10         10          5         8             6   
2020                      4          8          3         9             6   
2030                      4          7          4         2             8   
2040                      4          1          5         8             9   
2050                      6          7          1         2             7   
2060                      1          1          6         8             6   
2070                      4          2          5         2             0   
2080                      9          2          8         8             3   
2090                      1          8          6         7             2   
2100                      2          8          8         6             6   

MODEL                                                                         \
SCENARIO                                                                       
VARIABLE   CFC114_EMIS   CFC115_EMIS   CFC11_EMIS   CFC12_EMIS  CH3CCL3_EMIS   
TODO               SET           SET          SET          SET           SET   
UNITS    kt CFC-114/yr kt CFC-115/yr kt CFC-11/yr kt CFC-12/yr kt CH3CCl3/yr   
REGION           WORLD         WORLD        WORLD        WORLD         WORLD   
YEAR                                                                           
2015                 2             5           10            4             1   
2020                10            10            1            7             6   
2030                 0             5            7            7             2   
2040                 0             4            3            5             0   
2050                 1            10            0            7             7   
2060                10             5           10            0             1   
2070                10             7            3            0             0   
2080                 2             7            7            6             7   
2090                 9             3            6            8             4   
2100                 2             7            4            0            10   

MODEL       ...             NAME-MODEL-2                                \
SCENARIO    ...    SCENARIO-NAME-HYPHENS                                 
VARIABLE    ...            HCFC142B_EMIS   HCFC22_EMIS        HFC_EMIS   
TODO        ...                      SET           SET             SET   
UNITS       ...          kt HCFC-142b/yr kt HCFC-22/yr Mt CO2-equiv/yr   
REGION      ...                    WORLD         WORLD           WORLD   
YEAR        ...                                                          
2015        ...                        1             3               8   
2020        ...                        2             6               5   
2030        ...                        4             0               3   
2040        ...                        9             0               7   
2050        ...                        8             7               6   
2060        ...                        1             4               3   
2070        ...                        7             3               4   
2080        ...                        8             8               9   
2090        ...                        6             5               8   
2100        ...                        3             4               2   

MODEL                                                                 